# Evasion exercise

In this notebook, you will use SecML to implement adversarial attacks against a given neural network.
[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/zangobot/teaching_material/blob/HEAD/extra-EvasionExercise.ipynb)

In [ ]:
try:
    import secml
except ImportError:
    %pip install secml[foolbox]

In [2]:
import torch
from torch import nn
from torch import optim
from secml.ml.classifiers import CClassifierPyTorch

random_state = 999


# creation of the multiclass classifier
class Net(nn.Module):
    def __init__(self, n_features, n_hidden, n_classes):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(n_features, n_hidden)
        self.fc2 = nn.Linear(n_hidden, n_classes)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x


n_features = 2
n_classes = 3

# torch model creation
net = Net(n_features=n_features, n_classes=n_classes, n_hidden=100)

loss_function = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

# wrap torch model in CClassifierPyTorch class
clf = CClassifierPyTorch(model=net,
                         loss=loss_function,
                         optimizer=optimizer,
                         input_shape=(n_features,),
                         random_state=random_state)

In [ ]:
n_features = 2  # number of features
n_samples = 1250  # number of samples
centers = [[-2, 0], [2, -2], [2, 2]]  # centers of the clusters
cluster_std = 0.8  # standard deviation of the clusters
n_classes = len(centers)  # number of classes

from secml.data.loader import CDLRandomBlobs

dataset = ...  # TODO  instantiate a CDLRandomBlobs dataset as done in previous tutorials

n_tr = 1000  # number of training set samples
n_ts = 250  # number of test set samples

# split in training and test
from secml.data.splitter import CTrainTestSplit

splitter = CTrainTestSplit(
    train_size=n_tr, test_size=n_ts, random_state=random_state)
tr, ts = splitter.split(dataset)

# normalize the data
from secml.ml.features import CNormalizerMinMax

nmz = CNormalizerMinMax()
tr.X = nmz.fit_transform(tr.X)
ts.X = nmz.transform(ts.X)

In [ ]:
clf = ...  # TODO fit the classifier using SecML

y_pred = ...  # TODO  compute predictions on test set

from secml.ml.peval.metrics import CMetricAccuracy

accuracy = CMetricAccuracy().performance_score(y_true=ts.Y, y_pred=y_pred)
print("Accuracy on test set: {:.2%}".format(accuracy))


In [ ]:
%matplotlib inline
from secml.figure import CFigure

fig = CFigure()
fig.sp.plot_ds(tr)
fig.sp.plot_decision_regions(clf, plot_background=True,
                             n_grid_points=200)
fig.show()

In [ ]:
from secml.adv.attacks.evasion import CFoolboxPGDL2

y_target = None
index = 0
x0, y0 = ts.X[index, :], ts.Y[index]

# Attack parameters
lb = 0  # lower bound of the input space
ub = 1  # upper bound of the input space
epsilon = 0.03  # perturbation budget
abs_stepsize = 0.01  # stepsize of the attack
steps = 100  # number of iterations

pgd_attack =  # TODO: instantiate the CFoolboxPGDL2 to create an untargeted attack (y_target = None) against the trained clf

y_adv_pred, _, adv_ds_pgd, _ =  # TODO: run the attack on the sample (x0, y0)

print("Original x0 label: ", y0.item())
print("Adversarial example label (PGD-L2): ", y_adv_pred.item())

# Security Evaluations

Testing one single perturbation budget is not enough to understand the robustness of your machine learning classifier.
Hence, a fair analysis consists of a *security evaluation* of the model, by considering more perturbation budgets, and hence quantify the performance of the target systematically against stronger and stronger attacks.
To do so, we first set the number of parameters we want to test (in this case, `epsilons`) and we compute attacks by varying the perturbation size.

In [ ]:
epsilons = # create list of epsilons to use in the security evaluation
robust_accuracies = []
metric = CMetricAccuracy()
X, y = ts.X[:30, :], ts.Y[:30]

for eps in epsilons:
    pgd_attack =  # TODO: instantiate the CFoolboxPGDL2 to create an untargeted attack (y_target = None) against the trained clf with the correct epsilon
    y_adv_pred, _, _, _ =  # TODO: run the attack on the X, y set of samples
    accuracy = metric.performance_score(y, y_adv_pred)
    robust_accuracies.append(accuracy)

In [ ]:
%matplotlib inline
fig = CFigure(height=5, width=6)
fig.sp.plot(epsilons, robust_accuracies)
fig.show()